In [3]:
# opened w closed bas bey glitch
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(eye):
    # Apply Canny edge detection
    edges = cv2.Canny(eye, 30, 70)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(edges, 30, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to remove noise and smooth the image
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If there are contours, the eyes are considered open
    if contours:
        return "Open"
    else:
        return "Closed"

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Constants for eye blink detection
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Check for blinks
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

        # Detect eye status using Canny edge detection, thresholding, and morphology
        left_eye_status = detect_eye_status(leftEye)
        right_eye_status = detect_eye_status(rightEye)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

        # Display eye status on the frame
        cv2.putText(frame, "Left Eye: {}".format(left_eye_status), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, "Right Eye: {}".format(right_eye_status), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#         cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#         cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [2]:
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(left_eye, right_eye):
    # Calculate eye aspect ratios
    leftEAR = eye_aspect_ratio(left_eye)
    rightEAR = eye_aspect_ratio(right_eye)

    # Compare eye aspect ratios to check if they have similar relations
    ear_threshold = 0.05  # Adjust the threshold as needed
    ear_difference = abs(leftEAR - rightEAR)
    
    if ear_difference < ear_threshold:
        # Analyze color information to check if eyes are bright
        left_eye_mean_color = np.mean(left_eye)
        right_eye_mean_color = np.mean(right_eye)
        brightness_threshold = 150  # Adjust the threshold as needed

        if left_eye_mean_color > brightness_threshold and right_eye_mean_color > brightness_threshold:
            return "Awake"
        else:
            return "Closed"
    else:
        return "Closed"

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Constants for eye blink detection
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        # Apply adaptive histogram equalization
        leftEye = cv2.equalizeHist(leftEye)
        rightEye = cv2.equalizeHist(rightEye)

        # Check for blinks
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        if (leftEAR + rightEAR) / 2 < EYE_AR_CONSEC_FRAMES:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

        # Detect eye status using EAR and color information
        eye_status = detect_eye_status(leftEye, rightEye)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))

        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

        # Display eye status on the frame
        cv2.putText(frame, "Eye Status: {}".format(eye_status), (x + 10, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#         cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#         cv2.putText(frame, "EAR: {:.2f}".format((leftEAR + rightEAR) / 2), (x + 10, y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils
import time

def is_eye_closed(eye):
    # Check if eye region is empty or not detected
    if eye is None or eye.size == 0:
        return False  # Return false if eye is not detected or image is empty

    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)
    equalized_eye = cv2.equalizeHist(blurred_eye)
    _, thresh_eye = cv2.threshold(equalized_eye, 70, 255, cv2.THRESH_BINARY_INV)
    
    white_pixels = cv2.countNonZero(thresh_eye)
    white_area_percentage = (white_pixels / (thresh_eye.size)) * 100
    
    return white_area_percentage > 40  # Threshold value might need adjustment based on lighting and skin tone

# Load Haar Cascade Classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize and start the video stream
vs = VideoStream(src=0).start()
time.sleep(2.0)  # Give some time for the camera to warm-up

while True:
    frame = vs.read()
    
    # If frame could not be grabbed, then the stream might be closed or not accessible
    if frame is None:
        print("Failed to grab frame from camera. Check camera connection and index.")
        break

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_color = frame[y:y + h, x:x + w]
        roi_gray = gray[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_color[ey:ey + eh, ex:ex + ew]
            closed = is_eye_closed(eye)
            eye_status = 'Closed' if closed else 'Open'
            
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Eye Open or Closed Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the video stream and close windows
vs.stop()
cv2.destroyAllWindows()


In [3]:
#in this enhanced code:

#The is_eye_closed function now uses Canny edge detection and morphological transformations for better accuracy in detecting the closed state of eyes.
#The use of contour area is a simplistic way to determine if the eye is closed based on the assumption that closed eyes will produce fewer/smaller contours than open eyes.
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils
import time

def is_eye_closed(eye):
    """
    Determine if the eye is closed based on contour analysis of the thresholded eye region.
    
    Parameters:
    - eye (numpy.ndarray): The image region containing an eye.
    
    Returns:
    - bool: True if the eye is determined to be closed, otherwise False.
    """
    # Check if eye region is empty or not detected to avoid processing errors
    if eye is None or eye.size == 0:
        return False  

    # Convert eye region to grayscale and apply Gaussian blur to reduce noise and improve contour detection
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)

    # Use Canny edge detection to highlight edges in the eye region
    edges_eye = cv2.Canny(blurred_eye, 30, 70)

    # Apply morphological transformations to close gaps in the edges, helping in contour detection
    kernel = np.ones((3, 3), np.uint8)
    closed_eye = cv2.morphologyEx(edges_eye, cv2.MORPH_CLOSE, kernel)

    # Find contours in the edge-detected eye region to analyze the structure of the eye
    contours, _ = cv2.findContours(closed_eye, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Analyze contours; if few or small contours are found, it's likely the eye is closed
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)

        # Threshold for contour area to determine closed state might need adjustment for accuracy
        return area < 20
    return False

# Load Haar Cascade Classifiers for detecting faces and eyes
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize and start the video stream
vs = VideoStream(src=0).start()
time.sleep(2.0)  # Wait for camera to initialize

while True:
    # Read a frame from the video stream
    frame = vs.read()

    # Check if frame is successfully captured, otherwise break the loop
    if frame is None:
        print("Failed to grab frame from camera. Check camera connection and index.")
        break

    # Resize frame for faster processing and convert to grayscale for face detection
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame using Haar cascades
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest (ROI) where the face is detected
        roi_color = frame[y:y + h, x:x + w]
        roi_gray = gray[y:y + h, x:x + w]

        # Detect eyes within the face ROI using Haar cascades
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

        for (ex, ey, ew, eh) in eyes:
            # Extract eye region from the face ROI
            eye = roi_color[ey:ey + eh, ex:ex + ew]

            # Determine if the eye is closed or open
            closed = is_eye_closed(eye)
            eye_status = 'Closed' if closed else 'Open'

            # Draw a rectangle around the eye and display its status
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame with the eye status annotations
    cv2.imshow('Eye Open or Closed Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the video stream and close all windows
vs.stop()
cv2.destroyAllWindows()
